In [4]:
import numpy as np
from itertools import combinations

I12 = np.eye(12, dtype=int)                     #kreiramo identitetsku matricu 12x12#
P = np.array([
        [1,1,1,0,1,1,1,0,0,0,1,0],
        [1,0,1,1,0,1,1,1,0,0,0,1],
        [1,1,0,1,1,0,1,1,1,0,0,0],
        [1,0,1,0,1,1,0,1,1,1,0,0],
        [1,0,0,1,0,1,1,0,1,1,1,0],
        [1,0,0,0,1,0,1,1,0,1,1,1],
        [1,1,0,0,0,1,0,1,1,0,1,1],
        [1,1,1,0,0,0,1,0,1,1,0,1],
        [1,1,1,1,0,0,0,1,0,1,1,0],
        [1,0,1,1,1,0,0,0,1,0,1,1],
        [1,1,0,1,1,1,0,0,0,1,0,1],
        [0,1,1,1,1,1,1,1,1,1,1,1]
    ], dtype=int)                               #kreiramo matricu P#
G = np.hstack((I12, P))                         #funkcija koja kreira generatorsku matricu G tako što na identitetsku matricu horizontalno doda matricu P#

def encode_golay(data, G):                      #funkcija za kodiranje sekvence data Golay kodom#
    return np.dot(data, G) % 2                  #matrično se množi vektor data dimenzija (1,12) sa matricom G dimenzija (12,24) i dobija se kodirani vektor dimenzija (1,24),
                                                #tako što se na poziciji gdje je rezultat matričnog množenja neparan broj postavlja 1 a ako je paran postavlja 0#
def decode_golay(received, G):                  #funkcija za dekodiranje sekvence koja je primljena#
    n = G.shape[1]                              #broj kolona matrice G#
    k = G.shape[0]                              #broj redova matrice G#
    H = np.hstack((P.T, np.eye(12, dtype=int)))         #kreiramo matricu H (parity-check matricu) na osnovu koje zaključujemo da li postoji greška kod primljene sekvence i gdje se ona dogodila#
    syndrome = np.dot(received, H.T) % 2                #koristimo matricu H da dobijemo sindrom na osnovu primljene sekvence#

    if not np.any(syndrome):                            #ukoliko sindrom ima sve 0, onda ne postoji greška pa vraćamao primljenu sekvencu#
        return received[:k], received
    corrected = received.copy()                         #kopiramo primljenu sekvencu u corrected#
    for num_errors in range(1, 4):                      #broj grešaka koje se mogu ispraviti može biti 1,2 i 3#
        for error_positions in combinations(range(n), num_errors):   #provjerava sve moguće kombinacije grešaka, prvo ako imamo 1 grešku,onda 2, pa 3#
            error_vector = np.zeros(n, dtype=int)       #vektor grešaka se postavlja na 0 i dužine je kao i svaki red matrice G tj. 24#
            for pos in error_positions:                 
                error_vector[pos] = 1                   #postavljamo greške na odgovarajuće pozicije tj. postavljamo vrijednost 1 tamo gdje su se dogodile greške#
            if np.array_equal(syndrome, np.dot(error_vector, H.T) % 2):  #računa se sindrom za trenutnu kombinaciju grešaka i ako je on jednak prethodno izračunatom sindromu to znači da su greške u vektoru grešaka te koje su se pojavile u primljenoj sekvenci# 
                corrected = (corrected + error_vector) % 2      #ovo predstavlja xor operaciju gdje na mjestima gdje su se dogodile greške mijenjamo bit sa 0 na 1, odnosno sa 1 na 0 i tako ispravljamo grešku#
                return corrected[:k], corrected                 #vraća se ispravljena sekvenca#

    raise ValueError("Greška nije mogla biti ispravljena!")     #ako imamo više od 3 greške one ne mogu biti ispravljene#


data = np.array([1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1])           #sekvenca koja se šalje#
print("Ulazni podaci:", data)
encoded = encode_golay(data, G)                                 #kodiramo ovu sekvencu#
print("Kodirani podaci:", encoded)
corrupted = encoded.copy()                                      
corrupted[5] ^= 1                                               #mijenjamo bitove na pozicijama 5,8 i 11 odnosno simuliramo da su se dogodile 3 greške#
corrupted[8] ^= 1                                               
corrupted[11] ^= 1
print("Oštećeni podaci:", corrupted)                            #sekvenca sa greškama#
decoded, corrected = decode_golay(corrupted, G)                 #ispravljamo greške#
print(f"Dekodirani podaci: {decoded}")                          #dekodirana sekvenca od 12 bita koju smo poslali#
print(f"Ispravljeni podaci: {corrected}")                       #kompletna sekvenca od 24 bita#

Ulazni podaci: [1 0 1 0 1 1 0 0 1 1 0 1]
Kodirani podaci: [1 0 1 0 1 1 0 0 1 1 0 1 0 0 0 1 1 1 1 0 0 0 0 1]
Oštećeni podaci: [1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 1 1 1 0 0 0 0 1]
Dekodirani podaci: [1 0 1 0 1 1 0 0 1 1 0 1]
Ispravljeni podaci: [1 0 1 0 1 1 0 0 1 1 0 1 0 0 0 1 1 1 1 0 0 0 0 1]
